In [1]:
import pyvisa 
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime

        
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v




import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise
        
def set_Field(field=0, persistent=False):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise
        
def trigger_ac(ac, offs=0.005, amp=1e-6, duration=0.005, wait_after_arm=2):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(offs))
    ac.write("SOUR:WAVE:AMPL {}".format(amp))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(duration))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(wait_after_arm)
    ac.write("SOUR:WAVE:INIT")
    

    
def init_communication(amp=0.00005):
    print("check ac_1: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_1, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    
    print("check ac_2: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_2, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    """
    print("check ac_mid: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_mid, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)"""
    
    dc.apply_current()# Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = amp            # Sets the source current to 0.1 mA
    dc.enable_source() 
    print("enable dc current now, current:" + str(dc))
    print("check nano volt:")
    for i in range(10):
        print(volt.query("fetch?"))
        time.sleep(0.1)
    time.sleep(10)
    dc.shutdown()   
    print("dc current disabled")

    
def do_measurement_three_pulses(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "data/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in range(round):
        print("round: " + str(i+1))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=ac_current, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * ac_current, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""

rm = pyvisa.ResourceManager()
rm.list_resources()

('GPIB0::9::INSTR',)

In [ ]:
#switch
ac_6221_1_id = 14
ac_6221__2id = 3

dc_2400_id = 30
nanovolt_id = 8

ac_1 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221_1_id))
ac_2 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221__2id))

dc = Keithley2400("GPIB::{}".format(dc_2400_id))
volt = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))

In [2]:
nanovolt_id = 9
volt = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))

In [6]:
volt.query("fetch?")


'-5.28170067E-01\n'

In [20]:
init_communication(amp=1e-4)

check ac_1: offerset 50uA, amp1uA, duration 10s
check ac_2: offerset 50uA, amp1uA, duration 10s
enable dc current now, current:<pymeasure.instruments.keithley.keithley2400.Keithley2400 object at 0x00000255C237E820>
check nano volt:
+7.90352519E-07

+7.94559139E-07

+7.94559139E-07

-1.81679589E-05

-1.81679589E-05

-1.81688937E-05

-1.81688937E-05

-1.81686266E-05

-1.81686266E-05

-1.81730335E-05

dc current disabled


In [28]:
do_task = True
check_conn = False
anneal_at_high_T = False
anneal_current = 17*1e-3

In [29]:

if not do_task:
    raise

t_array = [300]



i = [16*1e-3,17*1e-3,18*1e-3]

pulse_durations = [0.001]
fields = [0]
wait_after_T = 1 # in minute




if check_conn:
    do_measurement_three_pulses(sample_id='LMB214_15nm_3nm__8leg_0dsegree_0T_switch_10sec_3sec__Noanneal'+ "G",
                   ac_current=10*1e-3, dc_current=10e-5, duration=0.001, 
                   round=2, point_per_round=4, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])
    raise

if anneal_at_high_T:
    set_T(350)
    do_measurement_three_pulses(sample_id='LMB214_15nm_3nm__8leg_0dsegree_0T_switch_10sec_3sec__Noanneal'+ "G",
               ac_current=anneal_current, dc_current=10e-5, duration=0.001, 
               round=4, point_per_round=10, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])

    
for t in t_array:
    print(datetime.datetime.now())
    print("set T to:" + str(t))
    print(datetime.datetime.now())
    print("start measurment:")
    if not t is None:
        set_T(t)
        time.sleep(wait_after_T*60)
    
    for c in i:
        for field in fields:
            for pulse_d in pulse_durations:
                print("current pulse:" + str(c))
                print("set field to:" + str(field))
                
                print(datetime.datetime.now())
                do_measurement_three_pulses(sample_id='LMB214_15nm_3nm__8leg_0dsegree_0T_switch_10sec_3sec_Noanneal'+ str(t) +"K" + str(field) + "G",
                               ac_current=c, dc_current=10e-5, duration=pulse_d, 
                               round=4, point_per_round=10, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])

do_task = False

2020-03-16 10:35:54.302347
set T to:300
2020-03-16 10:35:54.302347
start measurment:
current T 300.0046
current pulse:0.016
set field to:0
2020-03-16 10:38:11.090203
round: 1
current:ac_1
-3.119324595714286e-05
-3.128014490357143e-05
-3.131158312142857e-05
-3.1244427646428565e-05
-3.116649288571429e-05
-3.113303017142857e-05
-3.102547159642857e-05
-3.0909918832142856e-05
-3.080140605714286e-05
-3.0747820925e-05
current:ac_2
-3.076062379642857e-05
-3.077756583571429e-05
-3.0776650425e-05
-3.077364478214285e-05
-3.0787872835714284e-05
-3.0784708975e-05
-3.08049409e-05
-3.079847984285714e-05
-3.084107157142857e-05
-3.083407197857143e-05
round: 2
current:ac_1
-3.082522173214286e-05
-3.085399587857143e-05
-3.084668855714286e-05
-3.083907914642857e-05
-3.084849118928571e-05
-3.083749109642857e-05
-3.081033361428571e-05
-3.082134111071429e-05
-3.0806825639285716e-05
-3.079240917857142e-05
current:ac_2
-3.080858083571429e-05
-3.0835687892857136e-05
-3.084247188571429e-05
-3.084454358214286e-05

KeyboardInterrupt: 